In [22]:
import pandas as pd
import numpy as np
import os

In [23]:
def clean_up_data(filename, good_values, replacements, threshold):
    
    """
    Takes in a csv file and cleans up the data for percentage cover review. 
    Removes unwanted values and changes them to 'OTHER'.
    
    Parameters:
    filename (str): The name of the csv file to be cleaned.
    good_values (list): A list of strings. The values that are allowed. Has set default in main().
    threshold (int): The number of times a value must occur to be kept. Has set default in main().

    Returns: 
    pandas DataFrame: A cleaned up DataFrame with unwanted values replaced with 'OTHER'.
    """
    # read in the data into a pandas DataFrame
    data = pd.read_csv(filename)

    expected_column_names = ['DATE', 'Block', 'Plot', 'Transect', 'Canopy', 'First Foliar', 'Second Foliar', 'Third Foliar', 'Basal']
    if list(data.columns) != expected_column_names:
        raise ValueError('The columns in the csv file are not correct. They should be: DATE, Block, Plot, Transect, Canopy, First Foliar, Second Foliar, Third Foliar, Basal')

    # get rid of columns that are not needed and general clean up
    data.drop({'Transect', 'Basal'}, axis=1, inplace=True)

    # replace the data with the replacements dictionary
    data.replace(replacements, inplace=True)

    # fill in missing values with 'NONE'
    data.fillna(value='NONE', inplace=True)

    # combine canopy and each foliage level into one column
    foliage = pd.DataFrame(data[['Canopy', 'First Foliar', 'Second Foliar', 'Third Foliar']].stack())
    foliage.reset_index(drop=True, inplace=True)

    # remove rows that have values that are not in the good_values list
    mask = ~foliage.isin(good_values).any(axis=1)
    bad_values = foliage[mask]

    # replace values that accur less than 'threshold' times with 'OTHER'
    foliage_counts = foliage.value_counts()
    replace = foliage_counts[foliage_counts < threshold].index.tolist()     # this is a wierd list of tuples 
    replace = [item[0] for item in replace]                                 # changes it to a list of strings
    data.replace(replace, 'OTHER', inplace=True)    

    return data

In [24]:
def YearToPercent(data):

    """
    Takes in a cleaned up DataFrame and calculates the percentage of each foliage type per plot for each year.

    Parameters:
    data (pandas DataFrame): The cleaned up DataFrame.

    Returns:
    year_PlotSpecies (dict): A dictionary with the year as the key and a pandas DataFrame with the percentage of each foliage type per plot as the value
    """
    grouped = data.groupby(data['DATE'])
    yearly_dfs = {year: group for year, group in grouped}

    year_PlotSpecies = {}

    for year in yearly_dfs: 

        # melt our data by plot type
        melted_df = yearly_dfs[year].melt(id_vars = 'Plot', value_vars=['Canopy', 'First Foliar', 'Second Foliar', 'Third Foliar'], var_name='Foliage Level', value_name='Foliage Type')
        
        # Count occurrences of each foliage type per plot
        count_df = melted_df.groupby(['Plot', 'Foliage Type']).size().reset_index(name='Count')
        
        # Calculates the total counts per plot
        total_count = count_df.groupby('Plot')['Count'].sum().reset_index(name='Total')

        # Merge the total counts with the count_df
        count_df = count_df.merge(total_count, on='Plot')

        # Calculate the percentage of each foliage type per plot
        count_df['Percentage'] = count_df['Count'] / count_df['Total']*100

        # Pivot table to have the foliage types as columns
        count_df = count_df.pivot(index='Plot', columns='Foliage Type', values='Percentage').fillna(0).reset_index()

        count_df['Year'] = year

        year_PlotSpecies[year] = count_df

    return year_PlotSpecies

In [25]:
def TotalsByYear(data):
    ### HAS PROBLEMS ###
    """
    Takes in a cleaned up DataFrame and calculates the percentage of each foliage type for each year.

    Parameters:
    data (pandas DataFrame): The cleaned up DataFrame.

    Returns:
    percentage_df (dict): A dictionary with the year as the key and a pandas DataFrame with the percentage of each foliage type per plot as the value
    """
    # melt our data by year
    melted_df = data.melt(id_vars=['DATE'], value_vars=['Canopy', 'First Foliar', 'Second Foliar', 'Third Foliar'], var_name='Foliage Level', value_name='Foliage Type')

    # Group the df by year and each value is the count of each foliage type
    percentage_df = melted_df.groupby(['DATE', 'Foliage Type']).size().unstack(fill_value=0)

    # Divide the values by the total of each year
    percentage_df = percentage_df.div(percentage_df.sum(axis=1), axis=0) * 100

    # Reset the index
    percentage_df = percentage_df.reset_index()


    return percentage_df

In [26]:
def TotalsAll(data):
    """
    Takes in a cleaned up DataFrame and calculates the percentage of each foliage type for all years.

    Parameters:
    data (pandas DataFrame): The cleaned up DataFrame.

    Returns:
    percentage_df (pandas DataFrame): A pandas DataFrame with the percentage of each foliage type for all years.
    """

    # melt our data by year
    melted_df = data.melt(id_vars=['DATE'], value_vars=['Canopy', 'First Foliar', 'Second Foliar', 'Third Foliar'], var_name='Foliage Level', value_name='Foliage Type')
    
    # count occurrences of each foliage type per year
    count_df = melted_df.groupby(['Foliage Type']).size().reset_index(name='Count')

    # calculate the total counts per year
    total_count = count_df['Count'].sum()


    # calculate the percentage of each foliage type per year
    count_df['Percentage'] = count_df['Count'] / total_count * 100

    # get rid of the count column
    count_df.drop({'Count'}, axis=1, inplace=True)

    # pivot table to have the foliage types as columns
    percentage_df = count_df.transpose()

    # reindex
    percentage_df = percentage_df.reset_index(drop=True)

    # set headers as first row
    new_header = percentage_df.iloc[0] 
    percentage_df = percentage_df[1:]
    percentage_df.columns = new_header

    # set index to 'Percentage'
    percentage_df.index = ['Percentage']


    return percentage_df

In [27]:
def main(filename, 
                  good_values = ["ALAL", "ASTR", "BRRU","ERCI","SCAR","VULP","BRTE","CICU","LUPI","DEPI","STELL","BAMU",
                                "BAIL","ERIN","GULA","BORA","PLPA","IPPO","LEPE","ASNU","DAPU","ASTER","CAFL","LETE",
                                "SPAM","ANLA","CRPT","DICA","OEPR","ARPUF","GILIA","PHFR","PECTO","AMTE","DRCU","ERIAS",
                                "ERDE","PLOV","LASE","STMI2","AJO","CYAC","LATR","LELA","AMDU","THMO","YUBR","CORA",
                                "SIAN","EPNE","ACSP","GIST","ENFA","ALGR","LEVI","ALVI","ELEL","HAGL","HOVU","MEAL",
                                "POSE","PRLE","CETE","LACE","CAMI","ARTR","ALDE","SPIN","TRDU","NONE"], 
                  replacements = {'ABRTE' : 'BRTE', 'ALDI' : 'ALDE', 'BRDE' : 'BRTE', 'BRTR' :'BRTE', 'BRET' : 'BRTE',
                                  'CETE 5' : 'CETE', 'ELEL 5' : 'ELEL', 'LITTER' : 'NONE', 'NC' : 'NONE', 'NF' : 'NONE', '' : 'NONE'},
                  threshold = 50):
    """
    Takes in a csv file and cleans up the data for percentage cover review.
    
    Parameters:
    filename (str): The relative path of the csv file to be cleaned.
        Note: The csv must have columns 'DATE', 'Block', 'Plot', 'Transect', 'Canopy', 'First Foliar', 'Second Foliar', 'Third Foliar', 'Basal'
    good_values (list): A list of strings. The values that are allowed. Defaults to a list of Great Basin/Mojave Desert plants.
    replacements (dict): A dictionary of values that need to be replaced. Defaults to a dictionary of common mistakes.
    threshold (int): The number of times a value must occur to be kept. Defaults to 700.

    returns (csv): A csv file with percentage cover by species over year, site, block, and totals
    """
    data = clean_up_data(filename, good_values, replacements, threshold)
    
    ### get an individual table for each year describing percentage per plot and species ###
    year_PlotSpecies = YearToPercent(data)

    ### get a totals table for each year and species ###
    totals_year = TotalsByYear(data)

    ### get a total plot for each species over the entire study ###
    totals = TotalsAll(data)

    ### combine all the data into a csv file or excel file ###
    area = os.path.splitext(os.path.basename(filename))[0]
    output_folder_name = f'PercentCoverData_{area}'
    os.makedirs(output_folder_name, exist_ok=True)
    for year, df in year_PlotSpecies.items():
        df.to_csv(os.path.join(output_folder_name, f'{year}_CoverPercentage_ByPlot_{area}.csv'), index=False)

    df_list = []


    # combine every years data into one DataFrame
    for year, df in year_PlotSpecies.items():
        df_list.append(df)
    combined_df = pd.concat(df_list, ignore_index=True)
    combined_df = combined_df[['Year'] + [col for col in combined_df.columns if col != 'Year']]
    combined_df.fillna(0, inplace=True)

    # export as csv
    combined_df.to_csv(os.path.join(output_folder_name, f'CoverPercentage_{area}.csv'), index=False)
            
    totals_year.to_csv(os.path.join(output_folder_name, f'TotalsByYear_CoverPercentage_{area}.csv'), index=False)

    totals.to_csv(os.path.join(output_folder_name, f'Totals_CoverPercentage_{area}.csv'), index=False)

In [29]:
main('../2025_Jan_Cover_Data/Lytle_Cover_All.csv')